# 梯度

pytorch可以更好地将反向传播进行求导. 梯度就是函数变化的一个标量, 下图的箭头的长度就是梯度

![6](./assets/6.png)

$\theta_{t+1}=\theta_{t}-\alpha_t\Delta{f(\theta_{t})}$

就是对每一个$\theta$进行偏微分(其他看做常数对这个变量求导的那个)

- $y=wx+b$, w偏微分: $x$, b偏微分: $1$, 因此函数的梯度在任何位置的梯度就是$(x, 1)$
- $y=w^2x+b^2$, w偏微分: $2xw$, b偏微分: $2b$, 因此函数的梯度在任何位置的梯度就是$(2wx, 2b)$
- $y=e^wx+e^b$, w偏微分: $e^wx$, b偏微分: $e^b$, 因此函数的梯度在任何位置的梯度就是$(e^wx, e^b)$
- $f[y-(xw+b)]$, w偏微分: $2(y-xw-b)x$, b偏微分: $2(y-wx-b)$, 因此函数的梯度在任何位置的梯度就是$(2(y-xw-b)x, 2(y-wx-b))$
- $f(ylog(wx+b))$, 要知道$(log_ex)'=\frac{1}{x}$, w偏微分: $\frac{y}{wx+b}x$, b偏微分: $\frac{y}{wx+b}$, 因此函数的梯度在任何位置的梯度就是$(\frac{y}{wx+b}x, \frac{y}{wx+b})$

In [1]:
import torch


需要求导可以手动定义

In [2]:
x = torch.randn(3, 4, requires_grad=True)
x


tensor([[-0.4379,  0.3458,  0.2249, -0.8641],
        [-1.3174,  0.6861,  0.2538,  1.2211],
        [ 0.4777, -1.4014, -0.1084, -1.3358]], requires_grad=True)

In [3]:
# 或者使用pytorch定义
x.requires_grad = True
x


tensor([[-0.4379,  0.3458,  0.2249, -0.8641],
        [-1.3174,  0.6861,  0.2538,  1.2211],
        [ 0.4777, -1.4014, -0.1084, -1.3358]], requires_grad=True)

In [4]:
b = torch.randn(3, 4, requires_grad=True)
t = x+b
y = t.sum()
y


tensor(-2.4723, grad_fn=<SumBackward0>)

In [5]:
y.backward()  # 反向传播
b.grad  # 求导


tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## 神经网络的求偏导过程:
![image](./assets/1.png)

In [6]:
# 计算流程
x = torch.randn(1)
b = torch.randn(1, requires_grad=True)
w = torch.randn(1, requires_grad=True)
y = w*x
z = y+b
# y也需要, 因为w是在y的后面计算w必须先计算y
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad


(False, True, True, True)

In [7]:
x.is_leaf, b.is_leaf, w.is_leaf, y.is_leaf, z.is_leaf  # leaf节点, 是不是最后一个计算偏导的节点


(True, True, True, False, False)

In [8]:
z.backward(retain_graph=True)  # 如果不对梯度进行清零的话, 梯度默认会累加起来
w.grad


tensor([-0.1368])

In [9]:
b.grad


tensor([1.])

需要注意的是, 神经网络的训练一定要进行参数初始化, 这个对于找到最优解有至关重要的作用.

## Escape Minima

很多时候寻找到的是局部最小值, 于是我们可以添加一个动量让其继续向前移动一部分时间.

![7](./assets/7.png)

## 激活函数

In [10]:
# sigmoid, 容易出现梯度离散和梯度爆炸
a = torch.linspace(-100, 100, 10)
torch.sigmoid(a)


tensor([0.0000e+00, 1.6655e-34, 7.4564e-25, 3.3382e-15, 1.4945e-05, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])

In [11]:
# tanh 在rnn中用的比较多
a = torch.linspace(-100, 100, 10)
torch.tanh(a)


tensor([-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.])

In [12]:
a = torch.linspace(-1, 1, 10)
torch.relu(a)


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111, 0.3333, 0.5556, 0.7778,
        1.0000])

### Loss

MSE = $\sum[y-(wx+b)]^2$

L2-norm = $||y-(wx+b)||_2$

loss = $norm(y-(wx+b))^2$

两种方法根据loss对目标参数求导: 

- `torch.autograd.grad(loss,[w1,w2,...])`
- `loss.backward()`

In [23]:
from torch.nn import functional as F

"""
出现错误
RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

因为 requires_grad 需要配置为 True, pytorch需要在建立的时候建立可导信息
"""

# 自动计算梯度
x = torch.ones(1)
w = torch.full([1], 2.0)  # 需要使用float才可以进行梯度运算

w.requires_grad_()

# 计算loss
# 第一个参数是predict, 第二个参数是[w_1,w_2...]
# (1-2)^2 = 1
mse = F.mse_loss(torch.ones(1), w*x)
print(x, w, mse)

print("autograd:", torch.autograd.grad(mse, [w]))


tensor([1.]) tensor([2.], requires_grad=True) tensor(1., grad_fn=<MseLossBackward0>)
autograd: (tensor([2.]),)


除了上面使用autograd计算梯度的方法意外也可以使用backward()方法来计算梯度

backward方法会将路径上所有相关数据全部进行梯度运算

In [29]:
# 自动计算梯度
x = torch.ones(1)
w = torch.full([1], 2.0, requires_grad=True)

# 计算loss
mse = F.mse_loss(torch.ones(1), w*x)
print(mse)

"""
注意, grad是储存在每一个需要grad参数中的, 因此backward方法的结果是存储在w.grad参数中, 而不是backward方法的返回值
而backward方法在计算的时候是直接将grad累加进入每一个需要计算的参数中, 如果需要重新计算需要清空grad信息
"""
print(w.grad)  # None
mse.backward()
print("backward:", w.grad)  # 2
# 打印norm, 一般情况下会直接打印grad信息, 但是如果grad过大或者过小就可以打印norm信息来查看矩阵的norm
print("backward, w norm:", w.norm())


tensor(1., grad_fn=<MseLossBackward0>)
None
backward: tensor([2.])
backward, w norm: tensor(2., grad_fn=<LinalgVectorNormBackward0>)


### softmax

相比于sigmod函数, 我们不一定需要其给我们的最终结果

在最终结果中, 我们需要所有的概率和相加等于1, 因此我们可以使用softmax

![8](./assets/8.png)

softmax可以把更大的放的更大, 更小的放的更小. 类似金字塔效应.

In [53]:
a = torch.rand(3)  # 生成一个长度为3的随机float list
a.requires_grad_()
print(a, a.grad)

p = F.softmax(a, dim=0)  # 生成概率, 此时需要告诉softmax对哪一个维度进行softmax.
print(p)

"""
当输出不是标量时，调用.backward()就会出错
显示声明输出的类型作为参数传入,且参数的大小必须要和输出值的大小相同, 因此需要指定 p.data为输入数据
"""
p.backward(p, retain_graph=True)
# 第二次使用backward的时候汇报错误:
# RuntimeError: Trying to backward through the graph a second time (or directly access saved
# tensors after they have already been freed). Saved intermediate values of the graph are freed when you call
# .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the
# graph a second time or if you need to access saved tensors after calling backward.
# 意思就是需要指定 retain-graph=True 不去销毁计算梯度的图
p.backward(p)
print("A grad", a.grad)

# 重新建图
# 注意: backward函数的loss必须是一个dim=1长度为1的数, 不能有多个量. 损失必须是一个单一的定值, 不能是一个向量.
# 否则的话就是代码或者逻辑有问题
p = F.softmax(a, dim=0)
# 由于长度不为1, 所以对p的多个变量进行返回
print(torch.autograd.grad(p[1], [a], retain_graph=True))
print(torch.autograd.grad(p[2], [a]))


tensor([0.9376, 0.5373, 0.6736], requires_grad=True) None
tensor([0.4102, 0.2748, 0.3150], grad_fn=<SoftmaxBackward0>)
A grad tensor([ 0.0551, -0.0375, -0.0176])
(tensor([-0.1127,  0.1993, -0.0866]),)
(tensor([-0.1292, -0.0866,  0.2158]),)
